# デシル分析

デシル分析は、データセットを10等分し、各デシルが全体に占める割合や重要性を評価する手法です。経済学やマーケティングでよく用いられる手法です。データの分布や不平等を可視化し、戦略立案に役立てることが特徴です。
- 参考文献：https://qiita.com/NobuYoshi/items/2e44cbcc6df830c6538e

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/main/tabledata/marketing/decile_analysis.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd

### データセットの用意

In [20]:
dtypes = {
    'InvoiceNo': 'object',
    'StockCode': 'object',
    'Description': 'object',
    'Quantity': 'int8',
    'InvoiceDate': 'datetime64[ns]',
    'UnitPrice': 'float64',
    'CustomerID': 'object',
    'Country': 'object'
}

url = 'https://raw.githubusercontent.com/fuyu-quant/data-science-wiki/develop/datasets/OnlineRetail.csv'
df = pd.read_csv(url, dtype=dtypes, engine='python',encoding='shift_jis')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [21]:
data = df.query('Quantity >= 0 & UnitPrice >= 0').dropna(axis=0, subset=['CustomerID'])
data['TotalPrice'] = data['Quantity'] * data['UnitPrice']

### デシル分析

In [22]:
# 顧客ごとの合計金額を計算
decil = data[['CustomerID', 'TotalPrice']].groupby('CustomerID').sum().reset_index()
decil.head()

,CustomerID,TotalPrice
0,12346,26.00
1,12347,4060.40
2,12348,1546.68
3,12349,1757.55
4,12350,334.40


上位から10%ごとの値を計算

In [23]:
parties = decil['TotalPrice'].quantile(q=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).to_dict()
parties

{0.1: 151.812,
 0.2: 240.62800000000001,
 0.3: 340.95799999999997,
 0.4: 474.1040000000001,
 0.5: 656.69,
 0.6: 905.4079999999999,
 0.7: 1309.416,
 0.8: 1954.0280000000005,
 0.9: 3488.1160000000045}

Decileスコアを算出

In [24]:
def cal_decil(x, df):
    if x <= df[0.1]:
        return 1
    elif x <= df[0.2]:
        return 2
    elif x <= df[0.3]: 
        return 3
    elif x <= df[0.4]: 
        return 4
    elif x <= df[0.5]: 
        return 5
    elif x <= df[0.6]: 
        return 6
    elif x <= df[0.7]: 
        return 7
    elif x <= df[0.8]: 
        return 8
    elif x <= df[0.9]: 
        return 9
    else:
        return 10


decil['Decil_score'] = decil['TotalPrice'].apply(cal_decil, args=(parties,))
decil.head()

,CustomerID,TotalPrice,Decil_score
0,12346,26.00,1
1,12347,4060.40,10
2,12348,1546.68,8
3,12349,1757.55,8
4,12350,334.40,3


分割した各層ごとにどれぐらい売り上げに貢献しているのかを算出

In [17]:
decil = decil.sort_values('TotalPrice', ascending=False)
decil['Cumsum'] = decil['TotalPrice'].cumsum()
decil['Cumprod'] = decil['Cumsum'] / decil['Cumsum'].max()
decil

,CustomerID,TotalPrice,Decil_score,Cumsum,Cumprod
1683,14646,163828.72,10,163828.720,0.022457
1873,14911,135245.78,10,299074.500,0.040995
4183,18102,91425.60,10,390500.100,0.053527
1329,14156,73867.49,10,464367.590,0.063653
3757,17511,73435.60,10,537803.190,0.073719
...,...,...,...,...,...
1658,14609,12.60,1,7295299.774,0.999997
3004,16454,6.90,1,7295306.674,0.999998
1787,14792,6.20,1,7295312.874,0.999999
414,12875,5.95,1,7295318.824,0.999999


各階層が売り上げに占める割合．上位10%で55%近く占めている

In [18]:
decil[['Decil_score', 'Cumprod']].groupby('Decil_score').max() \
    .reset_index().sort_values('Decil_score', ascending=False)

,Decil_score,Cumprod
9,10,0.549453
8,9,0.702887
7,8,0.798170
6,7,0.862768
5,6,0.908003
4,5,0.941415
3,4,0.965139
2,3,0.982578
1,2,0.994014
0,1,1.000000
